# **Transformación de los datos**

**Elaborado por:** Jacqueline Fernández, Jazmín Fernández y Nina Odoux.

## **Métricas de calidad**

In [ ]:
# Importación de librerías

import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
# Lectura de los csv

driver_details = pd.read_csv('driver_details.csv')
driver_standings = pd.read_csv('driver_standings.csv')
race_details = pd.read_csv('race_details.csv')

In [ ]:
# Visualización de la cabecera de driver_details

driver_details.head()

,Car,Date,Driver,Grand Prix,PTS,Race Position,Year
0,Ferrari,21 May 1950,Alberto Ascari,Monaco,6.0,2,1950
1,Ferrari,04 Jun 1950,Alberto Ascari,Switzerland,0.0,DNF,1950
2,Ferrari,18 Jun 1950,Alberto Ascari,Belgium,2.0,5,1950
3,Ferrari,03 Sep 1950,Alberto Ascari,Italy,3.0,2,1950
4,Deidt Offenhauser,30 May 1950,Tony Bettenhausen,Indianapolis 500,1.0,DNF,1950


In [ ]:
# Visualización de la cabecera de driver_standings

driver_standings.head()

,Pos,Driver,Nationality,Car,PTS,DriverCode,Year
0,1,Nino Farina,ITA,Alfa Romeo,30.0,FAR,1950
1,2,Juan Manuel Fangio,ARG,Alfa Romeo,27.0,FAN,1950
2,3,Luigi Fagioli,ITA,Alfa Romeo,24.0,FAG,1950
3,4,Louis Rosier,FRA,Talbot-Lago,13.0,ROS,1950
4,5,Alberto Ascari,ITA,Ferrari,11.0,ASC,1950


In [ ]:
# Visualización de la cabezera de race_details

race_details.head()

,Pos,No,Driver,Car,Laps,Time/Retired,PTS,Year,Grand Prix,Detail,DriverCode
0,1,2,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1950,Great Britain,Race-Result,FAR
1,2,3,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,1950,Great Britain,Race-Result,FAG
2,3,4,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,1950,Great Britain,Race-Result,PAR
3,4,14,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,1950,Great Britain,Race-Result,GIR
4,5,15,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,1950,Great Britain,Race-Result,ROS


## **Métricas de calidad para cada csv**

In [ ]:
# Funciones de evaluación específicas para cada métrica de calidad
def evaluate_precision(column):
    if column.name == 'PTS':
        valid_numeric = column.apply(lambda x: isinstance(x, (int, float)) and x >= 0)
        return valid_numeric.mean() * 100
    return 100  # Placeholder general

def evaluate_lineage(column):
    return 100  # Placeholder, se requiere trazabilidad externa

def evaluate_semantics(column):
    if column.name == 'Year':
        current_year = datetime.now().year
        valid_years = column.apply(lambda x: isinstance(x, int) and x >= 1950 and x <= current_year)
        return valid_years.mean() * 100
    elif column.name == 'Driver':
        return column.notnull().mean() * 100  # Validación básica: No nulo
    return 100  # Placeholder para otras columnas

def evaluate_structure(column):
    if column.name == 'Date':
        valid_dates = pd.to_datetime(column, errors='coerce', dayfirst=True).notnull()
        return valid_dates.mean() * 100
    elif column.name == 'DriverCode':
        valid_codes = column.str.match(r'^\w{3}$')  # Verifica códigos de 3 caracteres
        return valid_codes.mean() * 100
    return 100  # Placeholder

def evaluate_completeness(column):
    return column.notnull().mean() * 100

def evaluate_consistency(column, df):
    if column.name == 'Year' and 'Date' in df.columns:
        valid_dates = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
        valid_years = valid_dates.dt.year == column
        return valid_years.mean() * 100
    return 100  # Placeholder

def evaluate_currency(column):
    if column.name == 'Date':
        max_age = pd.Timedelta(days=365 * 2)
        recent_data = pd.to_datetime(column, errors='coerce', dayfirst=True) > (datetime.now() - max_age)
        return recent_data.mean() * 100
    return 100  # Placeholder

def evaluate_timeliness(column):
    if column.name == 'Date':
        max_delay = pd.Timedelta(days=30)
        last_update = pd.to_datetime(column, errors='coerce', dayfirst=True)
        timeliness = (datetime.now() - last_update) < max_delay
        return timeliness.mean() * 100
    return 100  # Placeholder

def evaluate_reasonableness(column):
    if column.name == 'PTS':
        reasonable_points = column.apply(lambda x: isinstance(x, (int, float)) and x >= 0)
        return reasonable_points.mean() * 100
    elif column.name == 'Year':
        reasonable_years = column.apply(lambda x: x >= 1950 and x <= datetime.now().year)
        return reasonable_years.mean() * 100
    return 100  # Placeholder

def evaluate_identifiability(column):
    if column.name in ['DriverCode', 'Driver']:
        return column.is_unique * 100
    return 100  # Placeholder

# Evaluar métricas por archivo
def evaluate_quality(df):
    results = {}
    for column_name in df.columns:
        column = df[column_name]
        results[column_name] = {
            'Precisión': evaluate_precision(column),
            'Linaje': evaluate_lineage(column),
            'Semántica': evaluate_semantics(column),
            'Estructura': evaluate_structure(column),
            'Completitud': evaluate_completeness(column),
            'Consistencia': evaluate_consistency(column, df),
            'Moneda': evaluate_currency(column),
            'Puntualidad': evaluate_timeliness(column),
            'Razonabilidad': evaluate_reasonableness(column),
            'Identificabilidad': evaluate_identifiability(column),
        }
    return pd.DataFrame(results).T

# Evaluar cada dataset
results_driver_details = evaluate_quality(driver_details)
results_driver_standings = evaluate_quality(driver_standings)
results_race_details = evaluate_quality(race_details)

# Mostrar los resultados
print("Resultados para driver_details:")
print(results_driver_details)
print("\nResultados para driver_standings:")
print(results_driver_standings)
print("\nResultados para race_details:")
print(results_race_details)


Resultados para driver_details:
                Precisión  Linaje  Semántica  Estructura  Completitud  \
Car            100.000000   100.0      100.0  100.000000    99.964671   
Date           100.000000   100.0      100.0   13.248208   100.000000   
Driver         100.000000   100.0      100.0  100.000000   100.000000   
Grand Prix     100.000000   100.0      100.0  100.000000   100.000000   
PTS             99.949531   100.0      100.0  100.000000    99.949531   
Race Position  100.000000   100.0      100.0  100.000000    99.919249   
Year           100.000000   100.0      100.0  100.000000   100.000000   

               Consistencia  Moneda  Puntualidad  Razonabilidad  \
Car              100.000000   100.0        100.0     100.000000   
Date             100.000000     0.0          0.0     100.000000   
Driver           100.000000   100.0        100.0     100.000000   
Grand Prix       100.000000   100.0        100.0     100.000000   
PTS              100.000000   100.0        100.0

### **Comprobación de la ejecución de las métricas mediante asserts**

In [ ]:
# Verificar que las métricas se calculen para todas las columnas y datasets
assert not results_driver_details.empty, "Resultados de driver_details están vacíos"
assert not results_driver_standings.empty, "Resultados de driver_standings están vacíos"
assert not results_race_details.empty, "Resultados de race_details están vacíos"

# Verificar que cada métrica esté dentro del rango esperado (0 a 100)
for results in [results_driver_details, results_driver_standings, results_race_details]:
    for metric in ['Precisión', 'Linaje', 'Semántica', 'Estructura', 'Completitud',
                   'Consistencia', 'Moneda', 'Puntualidad', 'Razonabilidad', 'Identificabilidad']:
        assert results[metric].between(0, 100).all(), f"Valores fuera de rango en la métrica {metric} del dataset {results['Dataset'][0]}"

# Verificar que no falten métricas
expected_metrics = ['Precisión', 'Linaje', 'Semántica', 'Estructura', 'Completitud',
                    'Consistencia', 'Moneda', 'Puntualidad', 'Razonabilidad', 'Identificabilidad']
for results in [results_driver_details, results_driver_standings, results_race_details]:
    assert all(metric in results.columns for metric in expected_metrics), "Faltan métricas calculadas"

print("Todos los asserts se han ejecutado correctamente.")

Todos los asserts se han ejecutado correctamente.


### **Guardado de los resultados en HTML**

In [ ]:
# Guardar resultados en un archivo HTML
results_driver_details.to_html('results_driver_details.html')
results_driver_standings.to_html('results_driver_standings.html')
results_race_details.to_html('results_race_details.html')

# **Métricas a nivel global**

In [ ]:
# Funciones de evaluación para cada métrica de calidad
def evaluate_precision(column):
    if column.name in ['Monto_compra', 'PTS']:
        valid_numeric = column.apply(lambda x: isinstance(x, (int, float)) and x >= 0)
        return valid_numeric.mean() * 100
    return 100  # Placeholder general

def evaluate_lineage(column):
    return 100  # Placeholder, se requiere trazabilidad externa

def evaluate_semantics(column):
    if column.name == 'Codigo_postal':
        return column.astype(str).str.match(r'^\d{5}$').mean() * 100
    elif column.name == 'Estado':
        valid_states = ['CA', 'TX', 'NY', 'FL']
        return column.isin(valid_states).mean() * 100
    elif column.name == 'Year':
        current_year = datetime.now().year
        return column.apply(lambda x: isinstance(x, int) and 1950 <= x <= current_year).mean() * 100
    return 100  # Placeholder para otras columnas

def evaluate_structure(column):
    if column.name in ['Correo_electronico', 'DriverCode']:
        return column.astype(str).str.match(r'^[\w\.-]+@[\w\.-]+\.\w+$').mean() * 100 if column.name == 'Correo_electronico' else column.str.match(r'^\w{3}$').mean() * 100
    elif column.name == 'Telefono':
        return column.astype(str).str.match(r'^\d{10}$').mean() * 100
    elif column.name == 'Date':
        return pd.to_datetime(column, errors='coerce').notnull().mean() * 100
    return 100  # Placeholder

def evaluate_completeness(column):
    return column.notnull().mean() * 100

def evaluate_consistency(column, df):
    if column.name == 'Year' and 'Date' in df.columns:
        valid_dates = pd.to_datetime(df['Date'], errors='coerce')
        valid_years = valid_dates.dt.year == column
        return valid_years.mean() * 100
    return 100  # Placeholder

def evaluate_currency(column):
    if column.name in ['Fecha_registro', 'Fecha_ultima_actualizacion', 'Date']:
        max_age = pd.Timedelta(days=365 * 2)
        recent_data = pd.to_datetime(column, errors='coerce') > (datetime.now() - max_age)
        return recent_data.mean() * 100
    return 100  # Placeholder

def evaluate_timeliness(column):
    if column.name == 'Fecha_ultima_actualizacion':
        max_delay = pd.Timedelta(days=30)
        last_update = pd.to_datetime(column, errors='coerce')
        timeliness = (datetime.now() - last_update) < max_delay
        return timeliness.mean() * 100
    return 100  # Placeholder

def evaluate_reasonableness(column):
    if column.name in ['Edad', 'PTS', 'Monto_compra']:
        if column.name == 'Edad':
            reasonable_values = (column >= 0) & (column <= 120)
        else:
            reasonable_values = column.apply(lambda x: isinstance(x, (int, float)) and x >= 0)
        return reasonable_values.mean() * 100
    return 100  # Placeholder

def evaluate_identifiability(column):
    return column.is_unique * 100 if column.name in ['ID_cliente', 'Correo_electronico'] else 100

# Función para evaluar métricas por archivo
def evaluate_quality(df):
    results = {}
    for column_name in df.columns:
        column = df[column_name]
        results[column_name] = {
            'Precisión': evaluate_precision(column),
            'Linaje': evaluate_lineage(column),
            'Semántica': evaluate_semantics(column),
            'Estructura': evaluate_structure(column),
            'Completitud': evaluate_completeness(column),
            'Consistencia': evaluate_consistency(column, df),
            'Moneda': evaluate_currency(column),
            'Puntualidad': evaluate_timeliness(column),
            'Razonabilidad': evaluate_reasonableness(column),
            'Identificabilidad': evaluate_identifiability(column),
        }
    return pd.DataFrame(results).T

# Evaluar calidad en detalle
results_driver_details = evaluate_quality(driver_details)
results_driver_standings = evaluate_quality(driver_standings)
results_race_details = evaluate_quality(race_details)

# Métricas generales
general_metrics_driver_details = results_driver_details.mean()
general_metrics_driver_standings = results_driver_standings.mean()
general_metrics_race_details = results_race_details.mean()

# Mostrar resultados
print("\nMétricas generales para driver_details:")
print(general_metrics_driver_details)

print("\nMétricas generales para driver_standings:")
print(general_metrics_driver_standings)

print("\nMétricas generales para race_details:")
print(general_metrics_race_details)



Métricas generales para driver_details:
Precisión             99.992790
Linaje               100.000000
Semántica            100.000000
Estructura            87.606887
Completitud           99.976207
Consistencia          87.606887
Moneda                85.714286
Puntualidad          100.000000
Razonabilidad         99.992790
Identificabilidad    100.000000
dtype: float64

Métricas generales para driver_standings:
Precisión            100.000000
Linaje               100.000000
Semántica            100.000000
Estructura           100.000000
Completitud           99.902878
Consistencia         100.000000
Moneda               100.000000
Puntualidad          100.000000
Razonabilidad        100.000000
Identificabilidad    100.000000
dtype: float64

Métricas generales para race_details:
Precisión            100.000000
Linaje               100.000000
Semántica            100.000000
Estructura            99.996588
Completitud           99.908628
Consistencia         100.000000
Moneda         

### **Guardado de los resultados en HTML**

In [ ]:
# Guardar los resultados en un archivo HTML
# Convert the Series to a DataFrame before saving to HTML
pd.DataFrame(general_metrics_driver_details).to_html('general_metrics_driver_details.html')
pd.DataFrame(general_metrics_driver_standings).to_html('general_metrics_driver_standings.html')
pd.DataFrame(general_metrics_race_details).to_html('general_metrics_race_details.html')

### **Media de las métricas generales para driver_details, driver_standings y race_details**

In [ ]:
# Calcular la media de las métricas generales por csv
mean_driver_details = general_metrics_driver_details.mean()
mean_driver_standings = general_metrics_driver_standings.mean()
mean_race_details = general_metrics_race_details.mean()

# Mostrar resultados
print(f"Media general para driver_details: {mean_driver_details:.2f}")
print(f"Media general para driver_standings: {mean_driver_standings:.2f}")
print(f"Media general para race_details: {mean_race_details:.2f}")

Media general para driver_details: 96.09
Media general para driver_standings: 99.99
Media general para race_details: 99.99


### **Métricas generales para todos los datasets combinados**

In [ ]:
# Reutilizamos las funciones de evaluación previamente definidas

# Función para evaluar métricas por archivo
def evaluate_quality(df):
    results = {}
    for column_name in df.columns:
        column = df[column_name]
        results[column_name] = {
            'Precisión': evaluate_precision(column),
            'Linaje': evaluate_lineage(column),
            'Semántica': evaluate_semantics(column),
            'Estructura': evaluate_structure(column),
            'Completitud': evaluate_completeness(column),
            'Consistencia': evaluate_consistency(column, df),
            'Moneda': evaluate_currency(column),
            'Puntualidad': evaluate_timeliness(column),
            'Razonabilidad': evaluate_reasonableness(column),
            'Identificabilidad': evaluate_identifiability(column),
        }
    return pd.DataFrame(results).T

# Evaluar calidad para cada dataset
results_driver_details = evaluate_quality(driver_details)
results_driver_standings = evaluate_quality(driver_standings)
results_race_details = evaluate_quality(race_details)

# Combinar todos los resultados
combined_results = pd.concat([
    results_driver_details,
    results_driver_standings,
    results_race_details
])

# Calcular métricas generales combinadas
general_metrics_combined = combined_results.mean()

# Calcular calificación general basada en el promedio de métricas generales combinadas
single_score = general_metrics_combined.mean()

# Mostrar resultados
print("Resultados detallados combinados por columna:")
print(combined_results)

print("\nMétricas generales para todos los datasets combinados:")
print(general_metrics_combined)

print(f"\nCalificación general de calidad de datos: {single_score:.2f}")


Resultados detallados combinados por columna:
                Precisión  Linaje  Semántica  Estructura  Completitud  \
Car            100.000000   100.0      100.0  100.000000    99.964671   
Date           100.000000   100.0      100.0   13.248208   100.000000   
Driver         100.000000   100.0      100.0  100.000000   100.000000   
Grand Prix     100.000000   100.0      100.0  100.000000   100.000000   
PTS             99.949531   100.0      100.0  100.000000    99.949531   
Race Position  100.000000   100.0      100.0  100.000000    99.919249   
Year           100.000000   100.0      100.0  100.000000   100.000000   
Pos            100.000000   100.0      100.0  100.000000   100.000000   
Driver         100.000000   100.0      100.0  100.000000   100.000000   
Nationality    100.000000   100.0      100.0  100.000000   100.000000   
Car            100.000000   100.0      100.0  100.000000    99.320148   
PTS            100.000000   100.0      100.0  100.000000   100.000000   
Drive

# **Calificación final**

In [ ]:
print(f"\nCalificación general de calidad de datos: {single_score:.2f}")


Calificación general de calidad de datos: 98.90


# **Data Cleansing**

In [ ]:
def clean_driver_details(df):
    """
    Limpieza específica para el dataset driver_details.csv
    """
    print("\nLimpieza de driver_details.csv")

    registros_iniciales = len(df)

    # Estandarizar nombres de columnas
    df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

    # Eliminación de duplicados
    df.drop_duplicates(subset=['driver', 'date', 'grand_prix'], inplace=True)

    # Imputar valores nulos en 'Car'
    df['car'] = df['car'].fillna('Unknown').str.title()

    # Estandarizar formato de fechas
    def clean_date(date):
        try:
            return datetime.strptime(date, "%d %b %Y").strftime("%Y-%m-%d")
        except:
            return np.nan

    df['date'] = df['date'].apply(clean_date)

    # Limpiar posiciones de carrera
    def clean_position(pos):
        if pos in ['DNF', 'Retired'] or pd.isna(pos):
            return -1  # Marcador para "No Disponible"
        try:
            return int(pos)
        except:
            return -1

    df['race_position'] = df['race_position'].apply(clean_position)

    # Imputar valores nulos en 'PTS'
    df['pts'] = df['pts'].fillna(0).astype(float)

    # Validar tipo de datos
    df['year'] = df['year'].astype(int)

    registros_finales = len(df)

    print("\nResumen de la limpieza:")
    print(f"Registros iniciales: {registros_iniciales}")
    print(f"Registros finales: {registros_finales}")
    print("\nColumnas y sus tipos de datos:")
    for columna, tipo in df.dtypes.items():
        print(f"{columna}: {tipo}")

    return df

def clean_driver_standings(df):
    """
    Limpieza específica para el dataset driver_standings.csv
    """
    print("\nLimpieza de driver_standings.csv")

    registros_iniciales = len(df)

    # Estandarizar nombres de columnas
    df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

    # Eliminación de duplicados
    df.drop_duplicates(subset=['driver', 'year', 'car'], inplace=True)

    # Limpiar posiciones
    def clean_position(pos):
        if pos == 'NC' or pd.isna(pos):
            return -1  # Marcador para "No Disponible"
        try:
            return int(pos)
        except:
            return -1

    df['pos'] = df['pos'].apply(clean_position)

    # Imputar valores nulos en 'Car'
    df['car'] = df['car'].fillna('Unknown').str.title()

    # Validar tipo de datos
    df['pts'] = df['pts'].astype(float)
    df['year'] = df['year'].astype(int)

    registros_finales = len(df)

    print("\nResumen de la limpieza:")
    print(f"Registros iniciales: {registros_iniciales}")
    print(f"Registros finales: {registros_finales}")
    print("\nColumnas y sus tipos de datos:")
    for columna, tipo in df.dtypes.items():
        print(f"{columna}: {tipo}")

    return df

def clean_race_details(df):
    """
    Limpieza específica para el dataset race_details.csv
    """
    print("\nLimpieza de race_details.csv")

    registros_iniciales = len(df)

    # Estandarizar nombres de columnas
    df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

    # Eliminación de duplicados
    df.drop_duplicates(subset=['driver', 'year', 'grand_prix'], inplace=True)

    # Limpiar posiciones
    def clean_position(pos):
        if pos == 'NC' or pd.isna(pos):
            return -1  # Marcador para "No Disponible"
        try:
            return int(pos)
        except:
            return -1

    df['pos'] = df['pos'].apply(clean_position)

    # Limpiar valores de 'Time/Retired'
    df['time/retired'] = df['time/retired'].fillna('Unknown')

    # Limpiar valores de 'Laps'
    df['laps'] = df['laps'].fillna(0).astype(int)

    # Imputar valores nulos en 'Car'
    df['car'] = df['car'].fillna('Unknown').str.title()

    # Validar tipo de datos
    df['pts'] = df['pts'].astype(float)
    df['year'] = df['year'].astype(int)

    registros_finales = len(df)

    print("\nResumen de la limpieza:")
    print(f"Registros iniciales: {registros_iniciales}")
    print(f"Registros finales: {registros_finales}")
    print("\nColumnas y sus tipos de datos:")
    for columna, tipo in df.dtypes.items():
        print(f"{columna}: {tipo}")

    return df

def clean_data():
    """
    Función principal para limpiar todos los datasets
    """
    # Cargar datasets
    driver_details = pd.read_csv('driver_details.csv')
    driver_standings = pd.read_csv('driver_standings.csv')
    race_details = pd.read_csv('race_details.csv')

    # Limpiar cada dataset
    driver_details_cleaned = clean_driver_details(driver_details)
    driver_standings_cleaned = clean_driver_standings(driver_standings)
    race_details_cleaned = clean_race_details(race_details)

    # Guardar los resultados
    driver_details_cleaned.to_csv('driver_details_cleaned.csv', index=False)
    driver_standings_cleaned.to_csv('driver_standings_cleaned.csv', index=False)
    race_details_cleaned.to_csv('race_details_cleaned.csv', index=False)

    print("\nLimpieza completada y archivos guardados.")

# Ejecutar la limpieza
clean_data()



Limpieza de driver_details.csv

Resumen de la limpieza:
Registros iniciales: 19814
Registros finales: 19808

Columnas y sus tipos de datos:
car: object
date: object
driver: object
grand_prix: object
pts: float64
race_position: int64
year: int64

Limpieza de driver_standings.csv

Resumen de la limpieza:
Registros iniciales: 1618
Registros finales: 1618

Columnas y sus tipos de datos:
pos: int64
driver: object
nationality: object
car: object
pts: float64
drivercode: object
year: int64

Limpieza de race_details.csv

Resumen de la limpieza:
Registros iniciales: 23978
Registros finales: 23944

Columnas y sus tipos de datos:
pos: int64
no: int64
driver: object
car: object
laps: int64
time/retired: object
pts: float64
year: int64
grand_prix: object
detail: object
drivercode: object

Limpieza completada y archivos guardados.


# **Lectura de los dataframes limpios**

In [ ]:
driver_details_cleaned = pd.read_csv('driver_details_cleaned.csv')
driver_standings_cleaned = pd.read_csv('driver_standings_cleaned.csv')
race_details_cleaned = pd.read_csv('race_details_cleaned.csv')

In [ ]:
driver_details_cleaned.head()

,car,date,driver,grand_prix,pts,race_position,year
0,Ferrari,1950-05-21,Alberto Ascari,Monaco,6.0,2,1950
1,Ferrari,1950-06-04,Alberto Ascari,Switzerland,0.0,-1,1950
2,Ferrari,1950-06-18,Alberto Ascari,Belgium,2.0,5,1950
3,Ferrari,1950-09-03,Alberto Ascari,Italy,3.0,2,1950
4,Deidt Offenhauser,1950-05-30,Tony Bettenhausen,Indianapolis 500,1.0,-1,1950


In [ ]:
driver_standings_cleaned.head()

,pos,driver,nationality,car,pts,drivercode,year
0,1,Nino Farina,ITA,Alfa Romeo,30.0,FAR,1950
1,2,Juan Manuel Fangio,ARG,Alfa Romeo,27.0,FAN,1950
2,3,Luigi Fagioli,ITA,Alfa Romeo,24.0,FAG,1950
3,4,Louis Rosier,FRA,Talbot-Lago,13.0,ROS,1950
4,5,Alberto Ascari,ITA,Ferrari,11.0,ASC,1950


In [ ]:
race_details_cleaned.head()

,pos,no,driver,car,laps,time/retired,pts,year,grand_prix,detail,drivercode
0,1,2,Nino Farina,Alfa Romeo,70,2:13:23.600,9.0,1950,Great Britain,Race-Result,FAR
1,2,3,Luigi Fagioli,Alfa Romeo,70,+2.600s,6.0,1950,Great Britain,Race-Result,FAG
2,3,4,Reg Parnell,Alfa Romeo,70,+52.000s,4.0,1950,Great Britain,Race-Result,PAR
3,4,14,Yves Giraud-Cabantous,Talbot-Lago,68,+2 laps,3.0,1950,Great Britain,Race-Result,GIR
4,5,15,Louis Rosier,Talbot-Lago,68,+2 laps,2.0,1950,Great Britain,Race-Result,ROS
